# Figuring out Spark 2.0

aka the *Decimal Project*

### Notes/Discoveries

* dates are a real headache
* rdd's are still available, but the spark dataframe/set is now the preferred data structure, 
albeit a bit more complex.  For example, you don't create a dataframe or rdd from a random  
list, you create a range that is ALREADY a dataframe.


### References / Read

* http://www.agildata.com/apache-spark-rdd-vs-dataframe-vs-dataset/
* https://databricks.com/blog/2016/01/04/introducing-apache-spark-datasets.html
* http://blog.cloudera.com/blog/2015/07/how-to-do-data-quality-checks-using-apache-spark-dataframes/
* https://www.infoq.com/articles/apache-spark-sql
* http://blog.brakmic.com/data-science-for-losers-part-5-spark-dataframes/


In [1]:
##### CELL A #####
# Basic setup required in all notebooks
 
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import *

appName='archetest'
master='local[*]' #local spark-master

# Explicitly define python 2 since we have both 2 & 3 installed
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'

spark = (SparkSession
         .builder
         .master(master)
         .appName(appName)
         .getOrCreate())


In [2]:
##### CELL A-2 #####
# verify it works

# Old way
rdd = spark.sparkContext.parallelize(range(1000))
print rdd.takeSample(False, 5)

# Spark 2
# rdd's are still available, but the spark dataframe/set is now the preferred data structure, 
# albeit a bit more complex.  For example, you don't create a dataframe or rdd from a random  
# list, you create a range that is ALREADY a dataframe.
r = spark.range(10) #creates a dataframe/dataset
print type(r) ,r.take(5)


[801, 171, 954, 867, 441]
<class 'pyspark.sql.dataframe.DataFrame'> [Row(id=0), Row(id=1), Row(id=2), Row(id=3), Row(id=4)]


In [4]:
#stop here  # halt execution with an error

## Read & Parse a Clean Data File

### File Access

Read as text from local and HDFS stores. Use Spark 2.0 native csv functionality to read CSV & derive schema.

In [5]:
##### CELL READ A #####
# read local file into a dataset

datafile_local = "/home/jovyan/work/data/simpleSampleClean.csv"

#raw_dataRDD = sc.textFile(datafile_local)  # spark 1.6
raw_data1 = spark.read.text(datafile_local) # spark 2.0

print type(raw_data1)   #spark2 type is <class 'pyspark.sql.dataframe.DataFrame'>
print raw_data1.take(2)
raw_data1.printSchema()
#raw_data1.show()

<class 'pyspark.sql.dataframe.DataFrame'>
[Row(value=u'lineNum,aStr,aDate,anInt,aFloat,aDollarAmount,aNote'), Row(value=u'1,smart,1/1/2016,1,1.1, $1.10 ,good data')]
root
 |-- value: string (nullable = true)



In [6]:
##### CELL READ B #####
# Read file from HDFS

datafile_HDFS = r'hdfs://172.18.0.2:9000/user/root/testData/simpleSampleClean.csv'

raw_data2 = spark.read.text(datafile_HDFS)

print type(raw_data2)
print raw_data2.take(2)
raw_data2.printSchema()


<class 'pyspark.sql.dataframe.DataFrame'>
[Row(value=u'lineNum,aStr,aDate,anInt,aFloat,aDollarAmount,aNote'), Row(value=u'1,smart,2016/01/01,1,1.1, $1.10 ,good data')]
root
 |-- value: string (nullable = true)



In [7]:
##### CELL READ  C #####
# Read as default csv file with inferred header

raw_data2 = spark.read.csv(datafile_HDFS, header = "true")
print type(raw_data2)   # still a dataframe
raw_data2.printSchema()
raw_data2.show()

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- lineNum: string (nullable = true)
 |-- aStr: string (nullable = true)
 |-- aDate: string (nullable = true)
 |-- anInt: string (nullable = true)
 |-- aFloat: string (nullable = true)
 |-- aDollarAmount: string (nullable = true)
 |-- aNote: string (nullable = true)

+-------+------+----------+-----+------+-------------+---------+
|lineNum|  aStr|     aDate|anInt|aFloat|aDollarAmount|    aNote|
+-------+------+----------+-----+------+-------------+---------+
|      1| smart|2016/01/01|    1|   1.1|       $1.10 |good data|
|      2|  cows|  2016/1/2|    2|   2.2|       $2.20 |good data|
|      3|   moo|  2016/1/3|    3|   3.3|       $3.30 |good data|
|      4|longer|  2016/1/4|    4|   4.4|       $4.40 |good data|
+-------+------+----------+-----+------+-------------+---------+



In [8]:
##### CELL READ  D #####
# Open a simple file with pre-defined schema
# Drop for now; dates aren't worth the hassle

datafile_local = "/home/jovyan/work/data/simpleSampleClean_v2.csv"

schema = StructType([StructField("line", StringType(), False),
                     StructField("aStr", StringType(), False),
                     #StructField("aDate", StringType(), False), #
                     StructField("aDate", DateType(), False),
                     StructField("anInt", IntegerType(), False),
                     StructField("aFloat", DoubleType(), False),
                     StructField("aDollarAmount",StringType(),True),
                     StructField("aNote", StringType(), False)
                     ])  
raw_data3 = spark.read.csv(datafile_local,
                             schema = schema,
                             mode = 'DROPMALFORMED',  # PERMISSIVE DROPMALFORMED FAILFAST
                             sep = ',',
                             comment='#'
                            )

print type(raw_data3)
raw_data3.printSchema()
raw_data3.show()
#raw_data3.take(2)

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- line: string (nullable = true)
 |-- aStr: string (nullable = true)
 |-- aDate: date (nullable = true)
 |-- anInt: integer (nullable = true)
 |-- aFloat: double (nullable = true)
 |-- aDollarAmount: string (nullable = true)
 |-- aNote: string (nullable = true)

+----+----+-----+-----+------+-------------+-----+
|line|aStr|aDate|anInt|aFloat|aDollarAmount|aNote|
+----+----+-----+-----+------+-------------+-----+
+----+----+-----+-----+------+-------------+-----+



### Data Conversion (clean file)
Get Data into a useable format. Prototype regular expression parsing and user defined functions.

In [9]:
##### Cell Data Conversion A #####
# Data conversion/parsing setup

from pyspark.sql.functions import regexp_extract, regexp_replace, trim, col, lower, to_date, unix_timestamp, udf

# extraction patterns
re_us_currency = r'(\d*\.\d\d)'
re_integer = r'(\d*)'
re_float = r'(\d*\.?\d*)'


# Example user defined function to customize parsing
def Parse_Note(aStr):
    return "Noted: " + aStr

uMakeNote = udf(lambda x: Parse_Note(x),StringType())


In [10]:
##### Cell Data Conversion B #####

# Dates can be problematic
# to_date(regexp_replace(raw_data2.aDate,r'/',r'-')).alias('aDate')
#  -above works if date is in format yyyy/mm/dd
# unixTime soln from http://stackoverflow.com/questions/36948012/how-to-change-the-column-type-from-string-to-date-in-dataframes

# Transform into new dataset/dataframe with appropriate data types
parsed_data2 = (raw_data2
                .select(regexp_extract(raw_data2.lineNum,re_integer,1).alias('lineNum').cast("int"),
                        to_date(unix_timestamp(raw_data2.aDate, "yyyy/mm/dd").cast("timestamp")).alias('aDate'),
                        regexp_extract(raw_data2.anInt,re_integer,1).alias('anInt').cast("int"),
                        regexp_extract(raw_data2.aFloat,re_float,1).alias('aFloat').cast("float"),
                        regexp_extract(raw_data2.aDollarAmount,re_us_currency,1).alias("currency").cast("decimal(10,2)"),
                        uMakeNote(raw_data2.aNote).alias('aNote')
                  )
              )
parsed_data2.printSchema()
parsed_data2.show()


root
 |-- lineNum: integer (nullable = true)
 |-- aDate: date (nullable = true)
 |-- anInt: integer (nullable = true)
 |-- aFloat: float (nullable = true)
 |-- currency: decimal(10,2) (nullable = true)
 |-- aNote: string (nullable = true)

+-------+----------+-----+------+--------+----------------+
|lineNum|     aDate|anInt|aFloat|currency|           aNote|
+-------+----------+-----+------+--------+----------------+
|      1|2016-01-01|    1|   1.1|    1.10|Noted: good data|
|      2|2016-01-02|    2|   2.2|    2.20|Noted: good data|
|      3|2016-01-03|    3|   3.3|    3.30|Noted: good data|
|      4|2016-01-04|    4|   4.4|    4.40|Noted: good data|
+-------+----------+-----+------+--------+----------------+



In [11]:
##### Cell Data Conversion C #####
# validation check: there should be no null values
# Stay tuned!


## Access File with Imperfect Data
Read & parse a contrived sample of imperfect data

In [16]:
##### CELL Imperfect Data A #####
# Read file from HDFS

datafile_HDFS = r'hdfs://172.18.0.2:9000/user/root/testData/simpleSampleNotClean.csv'

raw_data = spark.read.csv(datafile_HDFS, header = "true")
raw_data.printSchema()
raw_data.show(truncate=False)

root
 |-- lineNum: string (nullable = true)
 |-- aStr: string (nullable = true)
 |-- aDate: string (nullable = true)
 |-- anInt: string (nullable = true)
 |-- aFloat: string (nullable = true)
 |-- aDollarAmount: string (nullable = true)
 |-- aNote: string (nullable = true)

+-------+------+---------+-----+------+-------------+----------------------------+
|lineNum|aStr  |aDate    |anInt|aFloat|aDollarAmount|aNote                       |
+-------+------+---------+-----+------+-------------+----------------------------+
|1      |smart |1/1/2016 |1    |1.1   | $1.10       |good data                   |
|2      |cows  |1/2/2016 |2    |2a    | $2.20       |float with text             |
|3      |moo   |1/3/2016 |3.3  |3.3   | $3.30       |int as float                |
|4      |longer|14/4/2016|4    |4.4   | $4.40       |bad date mo=14              |
|5      |than  |8/15/2016|5    |5.5   |abc          |text in currency            |
|6      |dogs  |9/12/2016|-6   |6.6   | $(99,999.00)|possible

In [19]:
##### CELL Imperfect Data B #####
parsed_data = (raw_data
                .select(regexp_extract(raw_data.lineNum,re_integer,1).alias('lineNum').cast("int"),
                        to_date(unix_timestamp(raw_data.aDate, "mm/dd/yyyy").cast("timestamp")).alias('aDate'),
                        regexp_extract(raw_data.anInt,re_integer,1).alias('anInt').cast("int"),
                        regexp_extract(raw_data.aFloat,re_float,1).alias('aFloat').cast("float"),
                        regexp_extract(raw_data.aDollarAmount,re_us_currency,1).alias("currency").cast("decimal(10,2)"),
                        uMakeNote(raw_data.aNote).alias('aNote')
                  )
              )
parsed_data.printSchema()
parsed_data.show(truncate=False)

root
 |-- lineNum: integer (nullable = true)
 |-- aDate: date (nullable = true)
 |-- anInt: integer (nullable = true)
 |-- aFloat: float (nullable = true)
 |-- currency: decimal(10,2) (nullable = true)
 |-- aNote: string (nullable = true)

+-------+----------+-----+------+--------+-----------------------------------+
|lineNum|aDate     |anInt|aFloat|currency|aNote                              |
+-------+----------+-----+------+--------+-----------------------------------+
|1      |2016-01-01|1    |1.1   |1.10    |Noted: good data                   |
|2      |2016-01-02|2    |2.0   |2.20    |Noted: float with text             |
|3      |2016-01-03|3    |3.3   |3.30    |Noted: int as float                |
|4      |2016-01-04|4    |4.4   |4.40    |Noted: bad date mo=14              |
|5      |2016-01-15|5    |5.5   |null    |Noted: text in currency            |
|6      |2016-01-12|null |6.6   |999.00  |Noted: possible out of range values|
+-------+----------+-----+------+--------+-------

Note above the regular expression parse, as designed, pulled the best match rather than report an error. These patterns could be improved. The date with month=14 was parsed as January; probably not desirable behavior.

In [23]:
##### CELL Imperfect Data C #####
# programmatically check for null values.
# We are assuming, of course, that null values are not permissible

bad_rows_df = parsed_data.filter(parsed_data.lineNum.isNull() |
                                 parsed_data.aDate.isNull() |
                                 parsed_data.anInt.isNull() |
                                 parsed_data.aFloat.isNull() |
                                 parsed_data.currency.isNull() |
                                 parsed_data.aNote.isNull()
                                 )

print "There are {} rows with null values: ".format(bad_rows_df.count())
bad_rows_df.show()

There are 2 rows with null values: 
+-------+----------+-----+------+--------+--------------------+
|lineNum|     aDate|anInt|aFloat|currency|               aNote|
+-------+----------+-----+------+--------+--------------------+
|      5|2016-01-15|    5|   5.5|    null|Noted: text in cu...|
|      6|2016-01-12| null|   6.6|  999.00|Noted: possible o...|
+-------+----------+-----+------+--------+--------------------+

